# Protenix-dock

In [ ]:
#| default_exp pxdock

TODO: 
get more conformers in sdf file, and test for virtual screening

## Install instruction

- [tutorial](https://zhuanlan.zhihu.com/p/1896530309580838652), install protenix-dock accordingly
- [official github](https://github.com/bytedance/Protenix-Dock)

## After install
>Test whether the notebook can import it successfully

if vscode:

```bash
export LD_LIBRARY_PATH=${CONDA_PREFIX}/lib:$LD_LIBRARY_PATH
```

If jupyterlab on a cloud, need to set secrete:

```python
import os
conda_lib_path = os.path.join(os.environ["CONDA_PREFIX"], "lib")
current_path = os.environ.get("LD_LIBRARY_PATH", "")
print(conda_lib_path,current_path)

# if in cloud, set the secrete according to below
os.environ['LD_LIBRARY_PATH'] ="/home/zeus/miniconda3/envs/cloudspace/lib:/opt/jupyter/envs/main/lib"
print(os.environ['LD_LIBRARY_PATH'])

```

## Setup

In [ ]:
#| export
# kdock
from kdock.core import *

# basics
import json,shutil
import os,sys,logging,contextlib
from datetime import datetime
from pathlib import Path

# import protenix-dock
try:
    from pxdock import ProtenixDock
    from pxdock.common.rmsd_calculator import write_ligand_to_sdf

    # Prevent multiprocessing-related errors in pxdock
    import multiprocessing as mp
    mp.set_start_method("spawn", force=True)

except ImportError as e:
    print(f"pxdock import failed: {e}")
    ProtenixDock = None
    write_ligand_to_sdf = None


pxdock import failed: libboost_python311.so.1.82.0: cannot open shared object file: No such file or directory


## One-time protenix-dock

In [ ]:
get_rec_lig('7OFF','VCB','protenix_test')

7OFF.pdb is detected!


('/teamspace/studios/this_studio/kdock/nbs/protenix_test/7OFF_receptor.pdb',
 '/teamspace/studios/this_studio/kdock/nbs/protenix_test/7OFF_lig.sdf')

In [ ]:
box = get_box('protenix_test/7OFF_lig.sdf',tolist=True)
box

[38.848, -26.77, 10.419, 14.652, 8.942, 12.509]

In [ ]:
#| export
@contextlib.contextmanager
def capture_output(log_path):
    with open(log_path, 'a') as log_file:  # Append mode prevents overwriting
        # Redirect stdout/stderr
        old_stdout_fd = os.dup(1)
        old_stderr_fd = os.dup(2)
        os.dup2(log_file.fileno(), 1)
        os.dup2(log_file.fileno(), 2)

        # Setup Python logger
        logger = logging.getLogger()
        logger.setLevel(logging.INFO)
        logger.handlers = [h for h in logger.handlers if not isinstance(h, logging.StreamHandler)]

        file_handler = logging.FileHandler(log_path)
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)

        try:
            yield
        finally:
            os.dup2(old_stdout_fd, 1)
            os.dup2(old_stderr_fd, 2)
            os.close(old_stdout_fd)
            os.close(old_stderr_fd)

In [ ]:
#| export
def get_protenix_dock(receptor_pdb, # pdb path
                      ligand_sdf, # sdf path
                      box:list, # center xyz + size xyz
                      out_dir='.', # output directory
                      num_walker=20, # number of Monte Carlo threads, the more walkers the more space to explore
                      mc_prune_energy_threshold=500, # maximum allowed energy for a pose; if exceeds, skipped
                      include_affinity=True, # include bscore
                      **kwargs):
    "Use protenix-dock to dock ligand sdf to receptor pdb using protenix-dock"
    Path(out_dir).mkdir(parents=True,exist_ok=True)
    
    dock_instance = ProtenixDock(receptor_pdb)
    box_center,box_size=box[:3],box[3:]
    dock_instance.set_box(box_center, box_size)
    
    log_path = Path(out_dir)/f'{datetime.now():%Y%m%d_%H%M%S}.log'
    with capture_output(log_path):
        # cache_dir = dock_instance.generate_cache_maps(spacing=0.175)
        result_dir = dock_instance.run_docking(ligand_sdf,out_dir=out_dir,num_walker=num_walker,mc_prune_energy_threshold=mc_prune_energy_threshold,include_affinity=include_affinity,**kwargs)
        # dock_instance.drop_cache_maps()
    
    print(f'Docking complete. Results in: {result_dir}')
    print('Cache dir:', cache_dir)

The function will output `-prepared-ligand/receptor.json` in the same directory as the receptor & ligand files

And output a `out.json` and log in the `out_dir`

In [ ]:
# %%time
# get_protenix_dock('protenix_test/7OFF_receptor.pdb',
#                   'protenix_test/7OFF_lig.sdf',
#                   box,
#                   out_dir='protenix_test/dock_result',
#                   num_walker=20
#                  )

## Protenix-vina
> Vina function in protenix-dock repository

In [ ]:
#| export
def get_protenix_vina_dock(receptor_pdb, # pdb path
                      ligand_sdf, # sdf path
                      box:list, # center xyz + size xyz
                      out_dir='.', # output directory
                      **kwargs):
    "Dock ligand sdf to receptor pdb using protenix-dock"
    Path(out_dir).mkdir(parents=True,exist_ok=True)
    
    dock_instance = ProtenixDock(receptor_pdb)
    box_center,box_size=box[:3],box[3:]
    dock_instance.set_box(box_center, box_size)
    
    log_path = Path(out_dir)/f'{datetime.now():%Y%m%d_%H%M%S}.log'
    with capture_output(log_path):
        result_dir = dock_instance.run_pose_opt(ligand_sdf,out_dir=out_dir,**kwargs)

    print(f'Docking complete. Results in: {result_dir}')

In [ ]:
# %%time
# get_protenix_vina_dock('protenix_test/7OFF_receptor.pdb',
#                   'protenix_test/7OFF_lig.sdf',
#                   box,
#                   out_dir='protenix_test/vina_result',
#                  )

## Output to sdf

In [ ]:
#| export
def json2sdf(json_path,
             sdf_path=None, # .sdf to be saved
             ):
    with open(json_path, "r") as f:
        json_out_data = json.load(f)

    best_pose = json_out_data["best_pose"]
    best_id = best_pose["index"]
    bscore = best_pose["bscore"]
    best_lig = json_out_data["poses"][best_id]
    xyz=best_lig["ligand"]["xyz"]
    if sdf_path is not None:
        write_ligand_to_sdf(json_out_data["mapped_smiles"], [xyz], sdf_path)
    return {'energy':best_lig["energy"],'pscore':best_lig["pscore"],'bscore':bscore}

In [ ]:
list(Path('protenix_test/dock_result').glob('*_out.json'))

[PosixPath('protenix_test/dock_result/7OFF_lig-prepared-ligand-0_out.json')]

In [ ]:
# json2sdf('protenix_test/dock_result/7OFF_lig-prepared-ligand-0_out.json',
#          sdf_path='protenix_test/predicted.sdf')

## End

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()